<a href="https://colab.research.google.com/github/daidrovo16/ETL-ArchivosPlanos/blob/main/ETL-ArchivosPlanos-Idrovo-Dario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2022-01-13 01:26:45--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip.1’

datasource.zip.1    100%[===================>]   4.15K  --.-KB/s    in 0s      

2022-01-13 01:26:45 (604 MB/s) - ‘datasource.zip.1’ saved [4249/4249]



In [ ]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
replace dealership_data/used_car_prices1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices1.csv  
replace dealership_data/used_car_prices2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices2.csv  
replace dealership_data/used_car_prices3.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices3.csv  
replace dealership_data/used_car_prices1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices1.json  
replace dealership_data/used_car_prices2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices2.json  
replace dealership_data/used_car_prices3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices3.json  
replace dealership_data/used_car_prices1.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dealership_data/used_car_prices1.xml  
rep

## **ESTABLECER LA RUTA DEL ARCHIVO DESTINO**

In [ ]:
tmpfile = "dealership_temp.tmp" # store all extracted data
logfile = "dealership_logfile.txt" # all event logs will be stored
targetfile = "dealership_transformed_data.csv" # transformed data is stored

**EXTRACCION CSV**

In [ ]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

**EXTRACCION JSON**

In [ ]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

**EXTRACCION XML**

In [ ]:
def extract_from_xml(file_to_process):

    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process) 
    root = tree.getroot() 
    for person in root: 
        car_model = person.find("car_model").text 
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text) 
        fuel = person.find("fuel").text 
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True) 
        return dataframe

**LLAMADA DE LA FUNCION DE EXTRACCION, HACIENDO USO DE LAS FUNCIONES DE CSV, JSON, XML.**

In [ ]:
def extract():
      extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) 
    #for csv files
      for csvfile in glob.glob("dealership_data/*.csv"):
          extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
    #for json files
      for jsonfile in glob.glob("dealership_data/*.json"):
          extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    #for xml files
      for xmlfile in glob.glob("dealership_data/*.xml"):
          extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
      return extracted_data

**FASE DE TRANSFORMACION**

In [ ]:
def transform(data):
       data['price'] = round(data.price, 2)
       return data

**SUBIR Y CARGAR LOS ARCHIVOS**

In [ ]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

In [ ]:
def log(message):
    timestamp_format="%H:%M:%S-%h-%d-%Y"
    #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f: f.write(timestamp + ',' + message + 'n')

**REGISTRO DE INICIO Y FINALIZACION DE LOS PROCESOS:**
> ETL

> TRANSFORMACION

> CARGA







In [ ]:
log("ETL Job Started")

In [ ]:
log("Extract phase Started")
extracted_data = extract() 
log("Extract phase Ended")

In [ ]:
log ("Transformation phase started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

In [ ]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [ ]:
log("ETL Job Ended")

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
